In [24]:
import sys, os
import re

In [28]:
def read_files(dirname):
    texts_target = []
    path = os.path.join('./nlp/for_weka/'+dirname)
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            if dirname == 'anekdots':
                texts_target += open(path+'/'+file).read().lower().split('***')
            else:
                texts_target += [open(path+'/'+file).read().lower()]
    return texts_target

In [29]:
#target_names = ['anekdots', 'teh_mol', 'izvest']
#читаем файлы и записываем их в массив, где эелементы - тексты
izvest = read_files('izvest')
teh_mol = read_files('teh_mol')
anekdots = read_files('anekdots')

In [41]:
# лемматизатор и стоп слова
from pymystem3 import Mystem
mystem = Mystem()
def lemm(st):
    if st == '':
        return ''
    else:
        return mystem.lemmatize(st)[0]
    
from stop_words import get_stop_words
from string import punctuation

stop_words = set(get_stop_words('ru'))

In [44]:
#лемматизатор текстов, который обратно "собирает" в структуру потекстового массиыв
i = 0
def lemm_text(target):
    new_trg = []
    for line in target:
        new_line = []
        for word in line.split():
            if word not in stop_words and word not in punctuation:
                new_line.append(lemm(word))
            #print(lemm(word))
        i += 1
        new_trg.append(('words='new_line, 'Tags='))
    return new_trg

In [45]:
#лемматихируем документы, делаем массив слов <= текст
lemm_izvest = lemm_text(izvest)
lemm_teh_mol = lemm_text(teh_mol)
lemm_anekdots = lemm_text(anekdots)

In [51]:
print(lemm_anekdots[:3])
#print(stop_words)
#составляем 

[['абстрактный', 'анекдот'], ['лететь', 'кирпич', '--\n', 'друг', 'ты', 'лететь', '--\n', 'равно', 'хороший', 'попадаться'], ['лететь', 'кирпич', 'старый', 'молодой', 'разговаривать', '--\n', 'давать', 'упасть', 'кто-нибудь', '--\n', 'никто', '--\n', 'смотреть', 'бригадир', 'идти', '--\n', 'каска', '--\n', 'что', 'смотреть', 'петровиииииииич', 'бригадир', 'подымать', 'голова', '--\n', 'ну', 'вы']]


In [100]:
#модель для векторизации, итоговый вектор длиной 50, 55 итераций
import gensim
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

#сливаем все слова из всех текстов в один массив, для подсчета словаря
all_texts = []
all_texts = lemm_anekdots
all_texts.extend(lemm_teh_mol)
all_texts.extend(lemm_izvest)
#размечаем документы так, как это нравится doc2vec
all_texts_trg = [gensim.models.doc2vec.TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(all_texts)]
print(all_texts_trg[:2])

[TaggedDocument(words=['абстрактный', 'анекдот'], tags=[0]), TaggedDocument(words=['лететь', 'кирпич', '--\n', 'друг', 'ты', 'лететь', '--\n', 'равно', 'хороший', 'попадаться'], tags=[1])]


In [102]:
#считаем вокубуляр и учимся строить вектора
model.build_vocab(all_texts_trg)
model.train(all_texts_trg)

60786460

In [104]:
def vectorize(doc, var):
    vect_doc = []
    for txt in doc:
        vect_doc.append((model.infer_vector(txt), var))
    return vect_doc

In [107]:
#векторизируем все тексты, добавляем номер варианта для классификатора, сливаем в одну переменную
vect_izvest = vectorize(lemm_izvest, '1')
vect_teh_mol = vectorize(lemm_teh_mol, '2')
vect_anekdots = vectorize(lemm_anekdots, '3')

all_vects = vect_anekdots
all_vects.extend(vect_teh_mol)
all_vects.extend(vect_izvest)

In [134]:
import pandas as pd

data = pd.DataFrame({'var': [],'vect':[]})
percent = 0
i = 0
for vec in all_vects:
    #print(vec[0])
    data.loc[i] = [vec[0], vec[1]]
    new_percent = round(len(data)/len(all_vects), 2)*100
    i += 1
    if new_percent > percent:
        print('Done:   '+ str(new_percent))
        percent = new_percent

Done:   1.0
Done:   2.0
Done:   3.0
Done:   4.0
Done:   5.0
Done:   6.0
Done:   7.000000000000001
Done:   8.0
Done:   9.0
Done:   10.0
Done:   11.0
Done:   12.0
Done:   13.0
Done:   14.000000000000002
Done:   15.0
Done:   16.0
Done:   17.0
Done:   18.0
Done:   19.0
Done:   20.0
Done:   21.0
Done:   22.0
Done:   23.0
Done:   24.0
Done:   25.0
Done:   26.0
Done:   27.0
Done:   28.000000000000004
Done:   28.999999999999996
Done:   30.0
Done:   31.0
Done:   32.0
Done:   33.0
Done:   34.0
Done:   35.0
Done:   36.0
Done:   37.0
Done:   38.0
Done:   39.0
Done:   40.0
Done:   41.0
Done:   42.0
Done:   43.0
Done:   44.0
Done:   45.0
Done:   46.0
Done:   47.0
Done:   48.0
Done:   49.0
Done:   50.0
Done:   51.0
Done:   52.0
Done:   53.0
Done:   54.0
Done:   55.00000000000001
Done:   56.00000000000001
Done:   56.99999999999999
Done:   57.99999999999999
Done:   59.0
Done:   60.0
Done:   61.0
Done:   62.0
Done:   63.0
Done:   64.0
Done:   65.0
Done:   66.0
Done:   67.0
Done:   68.0
Done:   69.0
Done

In [148]:
[[x for x in data['var'][i]] for i in range(0,len(data))]

[[0.18122488,
  -0.095988363,
  0.10037974,
  0.06728074,
  -0.18674172,
  -0.084628202,
  0.13572186,
  -0.093940936,
  0.097315654,
  -0.14605251,
  -0.0029977888,
  0.1035136,
  -0.070808992,
  -0.16232732,
  -0.074609302,
  0.16560052,
  -0.024006328,
  -0.1663657,
  -0.034219056,
  0.089738354,
  -0.013466926,
  -0.077106379,
  0.1885397,
  0.18461601,
  -0.08534734,
  0.038470045,
  0.12173507,
  -0.046178494,
  0.17604278,
  0.04200308,
  0.093646951,
  0.020497015,
  -0.11582255,
  -0.026887562,
  0.019011954,
  -0.1879645,
  0.3266857,
  0.19773445,
  -0.12048285,
  -0.0062494129,
  -0.096828245,
  0.26573715,
  0.13472472,
  -0.087820075,
  -0.12381712,
  0.057717431,
  -0.058176726,
  -0.14116228,
  -0.10219766,
  0.05880053],
 [-0.21936892,
  -0.31200084,
  -0.22946365,
  -0.028572679,
  -0.043075122,
  0.0059496472,
  0.25976864,
  -0.11419646,
  -0.31207091,
  -0.026131287,
  -0.2935535,
  0.027491866,
  -0.34101707,
  -0.28414837,
  0.04531179,
  -0.11402188,
  -0.003253

In [163]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn import metrics
 
clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=-1)

from sklearn.model_selection import cross_val_score

data = shuffle(data)
scores_acc = cross_val_score(clf, [[x for x in data['var'][i]] for i in range(0,len(data))], data['vect'], cv=5, scoring='accuracy')
#scores_prec = cross_val_score(clf, [[x for x in data['var'][i]] for i in range(0,len(data))], data['vect'], cv=5, scoring='average_precision')
scores_f1 = cross_val_score(clf, [[x for x in data['var'][i]] for i in range(0,len(data))], data['vect'], cv=5, scoring='f1_macro')#print(scores_acc)
#print(scores_prec) 
print(scores_f1)

#pfa = metrics.precision_recall_fscore_support(average='macro')
#cross_val_score(clf, [[x for x in data['var'][i]] for i in range(0,len(data))], data['vect'], cv=5, scoring=metrics.precision_recall_fscore_support(average='macro'))

/Users/Ksenia/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Ksenia/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Ksenia/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Ksenia/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[ 0.33013926  0.33013926  0.33013926  0.33013926  0.32896604]
